In [56]:
import json
import os
import random
import sys

import tensorflow as tf
import numpy as np
import pickle
from bert_serving.client import BertClient, ConcurrentBertClient
from tensorflow.estimator import BaselineClassifier
from tensorflow.python.estimator.canned.dnn import DNNClassifier
from tensorflow.python.estimator.run_config import RunConfig
from tensorflow.python.estimator.training import TrainSpec, EvalSpec, train_and_evaluate
import tensorflow_hub as hub

import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

tf.logging.set_verbosity(tf.logging.INFO)

In [57]:
batch_size = 128
num_parallel_calls = 1
bc = BertClient()

In [ ]:
max_seq_len = 200
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

encode_count = 0
def encode(chunk):
    global encode_count
    print('Chunk {}'.format(encode_count))
    encode_count += 1
    return bc.encode(chunk)
        
def get_encodes(data):
    # x is `batch_size` of lines, each of which is a json object
    features = np.array([])
    
    # TODO: modified
    text = []
    for x in data:
#         tokens = x[0].split(' ')
#         if len(tokens) > max_seq_len:
#             beginning = tokens[0:int(max_seq_len/2)]
#             end = tokens[int(-max_seq_len/2):]
#             new_review = '{} ... {}'.format(' '.join(beginning), ' '.join(end))
#             text.append(new_review.lower())
#         else:
        text.append(x[0].lower())
    features = np.concatenate([encode(chunk) for chunk in chunks(text, 256)])
    # randomly choose a label
    labels = [x[1] for x in data]
    return features, labels

def cache_data(data_dir, dest_dir, start_chunk, end_chunk):
    pos_files = os.listdir(os.path.join(data_dir, 'pos'))
    neg_files = os.listdir(os.path.join(data_dir, 'neg'))
        
    data = []
    for pos_file, neg_file in zip(pos_files, neg_files):
        with open(os.path.join(data_dir, 'pos', pos_file)) as f:
            review = f.readlines()[0].strip()
            data.append((review, 1))
        with open(os.path.join(data_dir, 'neg', neg_file)) as f:
            review = f.readlines()[0].strip()
            data.append((review, 0))
    chunk_num = -1
    chunk_size = 2048
    for chunk in chunks(data, chunk_size):
        chunk_num += 1
        if chunk_num < start_chunk:
            continue
        if chunk_num > end_chunk:
            break
        features, output = get_encodes(chunk)
        print('Wrote data_{:03d}.p'.format(chunk_num))
        with open(os.path.join(dest_dir, 'data_{:03d}.p'.format(chunk_num)), 'wb') as f:
            pickle.dump((features, output), f)

# input_fn_train = cache_data('/home/eugenet/final_project/data/aclImdb/train/', '/home/eugenet/final_project/cached_data/train_uncased_large_max200_frontback', 0, 14)
input_fn_eval = cache_data('/home/eugenet/final_project/data/aclImdb/test/', '/home/eugenet/final_project/cached_data/test_uncased_large_max200', 0, 14)

Chunk 0


In [15]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
    return bert.tokenization.FullTokenizer(
        vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0408 01:14:23.880115 140294087993152 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [33]:
from scipy.spatial.distance import cosine
cosine(bc.encode(['subwordinfo']), bc.encode(['infodorwsub']))

0.20648670196533203